<a href="https://colab.research.google.com/github/EricCui2005/Chuffin/blob/main/Chuffin_Baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title Imports
import numpy as np
import tensorflow as tf
import shutil
import random
import os
import glob
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import matplotlib.pyplot as plt
%matplotlib inline

import logging
logging.getLogger("tensorflow").setLevel(logging.ERROR)
tf.autograph.set_verbosity(0)

# Gooooogle Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


Mounted at /content/drive


In [ ]:
#@title Renaming and "Labeling" Chihuahua Images

# Directory with the chihuahua images
dir = '/content/drive/Shareddrives/Choofin/1124 Chihuahuas'
files = os.listdir(dir)

os.chdir('/content/drive/Shareddrives/Choofin/1124 Chihuahuas')

# Define a function to rename files
def rename_files():
    for i in range(len(files)):
        new_name = f"c_{i}"
        os.rename(files[i], new_name)

# Call the main function to rename files
rename_files()
files = os.listdir(dir)
print(files)

In [ ]:
#@title Renaming and "Labeling" Muffin Images

# Directory with the chihuahua images
dir = '/content/drive/Shareddrives/Choofin/muffin'
files = os.listdir(dir)

os.chdir('/content/drive/Shareddrives/Choofin/muffin')

# Define a function to rename files
def rename_files():
    for i in range(len(files)):
        new_name = f"m_{i}"
        os.rename(files[i], new_name)

# Call the main function to rename files
rename_files()
files = os.listdir(dir)
print(files)

In [ ]:
#@title Clearing a folder
# Specify the folder you want to clear
folder_path = '/content/drive/Shareddrives/Choofin/train'  # Change this to your folder path

# Check if the folder exists
if os.path.exists(folder_path) and os.path.isdir(folder_path):
    # List all files and directories in the folder
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        try:
            # If it is a file, delete it
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            # If it is a directory, delete it and all its contents
            elif os.path.isdir(file_path):
                # For deleting directories and their contents, you can use shutil.rmtree
                import shutil
                shutil.rmtree(file_path)
            print(f"{file_path} is deleted.")
        except Exception as e:
            print(f"Failed to delete {file_path}. Reason: {e}")
else:
    print(f"The specified path does not exist: {folder_path}")


In [ ]:
#@title Combining Dataset

source_folder = '/content/drive/Shareddrives/Choofin/1124 Chihuahuas'
destination_folder = '/content/drive/Shareddrives/Choofin/train'

# List all files and folders in the source folder
for item in os.listdir(source_folder):
    source_item = os.path.join(source_folder, item)
    destination_item = os.path.join(destination_folder, item)

    # Move each item to the destination folder
    if os.path.isfile(source_item):
        print(f"Copying {source_item}")
        shutil.copy(source_item, destination_item)

source_folder = '/content/drive/Shareddrives/Choofin/1088 Muffins'
destination_folder = '/content/drive/Shareddrives/Choofin/train'

# List all files and folders in the source folder
for item in os.listdir(source_folder):
    source_item = os.path.join(source_folder, item)
    destination_item = os.path.join(destination_folder, item)

    # Move each item to the destination folder
    if os.path.isfile(source_item):
        print(f"Copying {source_item}")
        shutil.copy(source_item, destination_item)


In [ ]:
#@title Creating the Label Set
files = os.listdir('/content/drive/Shareddrives/Choofin/train')
print(len(files))

# Training labels
label_set = []

for img in files:
  if(chr(ord(img[0])) == 'm'):
    label_set.append(0)
  elif(chr(ord(img[0])) == 'c'):
    label_set.append(1)

print(len(label_set))

In [ ]:
#@title Verifying Labels
print(len(files))
print(len(label_set))

for i in range(len(files)):
  if (files[i][0] == 'c'):
     print(label_set[i] == 1)
  elif (files[i][0] == 'm'):
     print(label_set[i] == 0)

In [ ]:
#@title Converting Images to (64, 64, 3) and loading X

import os
import numpy as np
from tensorflow.keras.preprocessing.image import img_to_array, load_img

def preprocess_images(folder_path, image_size=(64, 64)):
    images = []
    file_names = os.listdir(folder_path)

    for file_name in file_names:
        print(f"Converting {file_name}")
        # Load image
        img_path = os.path.join(folder_path, file_name)
        img = load_img(img_path, target_size=image_size)

        # Convert image to array and normalize
        img_array = img_to_array(img) / 255.0

        images.append(img_array)

    return np.array(images)

# Example usage
folder_path = '/content/drive/Shareddrives/Choofin/train'
processed_images = preprocess_images(folder_path)

print(processed_images.shape)  # Should show (number_of_images, 64, 64, 3)
X = processed_images


In [ ]:
#@title Loading the Data
from PIL import Image
import numpy
import os
directory = '/content/drive/Shareddrives/Choofin/train'

def load_data():
  X = []
  Y =  label_set # 1 means dog, 0 means muffin
  labels = np.array(Y)
  for file in os.listdir(directory):
    print(file)
    file_path = os.path.join(directory, file)
    img = Image.open(file_path)
    pix_val = list(img.getdata())
    pix_val_flat = [x for sets in pix_val for x in sets] # turns list of tuples into just a single list
    normalized_pix_val = [float(x / 255) for x in pix_val_flat]
    X.append(normalized_pix_val)
  data = np.array(X)
  return data, labels

X, Y = load_data()

In [ ]:
#@title Printing Shape
Y = np.array(label_set)

print(X.shape)
print(Y.shape)

In [ ]:
#@title Perceptron Model Architecture
from keras import layers
from keras import regularizers
model = Sequential(
    [
        tf.keras.Input(shape=(49152,)),    #specify input size
        Dense(64, activation='ReLU', kernel_regularizer=regularizers.L1(0.0001)),
        Dense(32, activation='ReLU', kernel_regularizer=regularizers.L1(0.00002)),
        Dense(16, activation='ReLU', kernel_regularizer=regularizers.L1(0.00004)),
        Dense(1,  activation='sigmoid')
    ], name = "my_model"
)

model.summary()
[layer1, layer2, layer3, layer4] = model.layers
print(model.layers[3].weights)


In [ ]:
#@title CNN Architecture
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential()
# Convolutional layer that learns 32 filters using a 3x3 kernel
model.add(Conv2D(32, (4, 4), activation='relu', input_shape=(64, 64, 3)))
model.add(MaxPooling2D((2, 2)))

# Additional convolutional layers
model.add(Conv2D(64, (4, 4), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (4, 4), activation='relu'))
model.add(MaxPooling2D((2, 2)))

# Flattening the 3D outputs to 1D before passing them to a Dense layer
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))  # Dropout to prevent overfitting

# Output layer for binary classification
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Model summary
model.summary()


In [23]:
#@title Compiling the Model
import keras

model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(0.0001),
    metrics=['accuracy', keras.metrics.AUC(name='auc')]
)

# model.fit(
#     X,Y,
#     batch_size=64,
#     epochs=80
# )

In [ ]:
#@title True vs Predicted Labels

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
# You do not need to modify anything in this cell

m, n, o, p = X.shape

fig, axes = plt.subplots(6,6, figsize=(6,6))
fig.tight_layout(pad=0.1,rect=[0, 0.03, 1, 0.92]) #[left, bottom, right, top]

for i,ax in enumerate(axes.flat):
    # Select random indices
    random_index = np.random.randint(m)

    # Select rows corresponding to the random indices and
    # reshape the image
    X_random_reshaped = X[random_index].reshape((64,64,3))

    # Display the image
    ax.imshow(X_random_reshaped)

    # Predict using the Neural Network
    prediction = model.predict(X[random_index].reshape(1,64,64,3))
    if prediction >= 0.5:
        yhat = 1
    else:
        yhat = 0

    # Display the label above the image
    ax.set_title(f"{Y[random_index]},{yhat}")
    ax.set_axis_off()
fig.suptitle("Label, yhat", fontsize=16)
plt.show()

In [ ]:
#@title Fitting the model for history purposes
history = model.fit(X, Y, validation_split=0.2, epochs=60, batch_size=32)

In [ ]:
#@title Graphing our model history
print(history.history.keys())
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# AUC plot
auc = model.history.history['auc']
val_auc = model.history.history['val_auc']

epochs = range(1, len(auc) + 1)

plt.plot(epochs, auc, 'bo', label='Training AUC')
plt.plot(epochs, val_auc, 'b', label='Validation AUC')
plt.title('Training and Validation AUC')
plt.xlabel('Epochs')
plt.ylabel('AUC')
plt.legend()

plt.show()

from sklearn.metrics import confusion_matrix

batch_size = 32
y_prediction = []

for i in range(0, len(X), batch_size):
    # Extract a batch of data
    batch_X = X[i:i+batch_size]

    # Predict for the batch
    batch_prediction = model.predict(batch_X)

    # Convert probabilities to class labels
    batch_y_pred = (batch_prediction >= 0.5).astype(int)

    # Collect the predictions for the batch
    y_prediction.extend(batch_y_pred)

y_pred = np.array(y_prediction)
cm = confusion_matrix(Y, y_pred)

print("Confusion Matrix:")
print(cm)


In [ ]:
#@title Analyzing Incorrect Predictions
false_positives = []
false_negatives = []

for i in range(len(Y)):
    if Y[i] == 1 and y_pred[i] == 0:
        false_negatives.append((X[i], Y[i], y_pred[i]))
    elif Y[i] == 0 and y_pred[i] == 1:
        false_positives.append((X[i], Y[i], y_pred[i]))

total_samples = len(Y)
false_negative_percentage = len(false_negatives) / total_samples * 100
false_positive_percentage = len(false_positives) / total_samples * 100

print("False Negative Percentage:", false_negative_percentage)
print("False Positive Percentage:", false_positive_percentage)

fig, axes = plt.subplots(6, 6, figsize=(6, 6))
fig.tight_layout(pad=0.1, rect=[0, 0.03, 1, 0.92])  # [left, bottom, right, top]

for i, (ax, (sample, true_label, _)) in enumerate(zip(axes.flat, false_positives)):
    sample_reshaped = sample.reshape((64, 64, 3))
    ax.imshow(sample_reshaped)

    prediction = model.predict(sample.reshape(1, 64, 64, 3))
    yhat = 1 if prediction >= 0.5 else 0

    ax.set_title(f"True: {true_label}, Predicted: {yhat}")
    ax.set_axis_off()

fig.suptitle("False Negatives: True Label vs. Predicted Label", fontsize=16)
plt.show()